In [9]:
from EEGModels import EEGNet
import scipy
import numpy as np
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import balanced_accuracy_score

In [10]:
folder_path = '../data/walk_instruction_raw/'
file_list = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.mat')]

file_list.sort()

print(file_list)

X = []
Y = []

for i, file in enumerate(file_list):
    x = scipy.io.loadmat(file)
    x = x['data']

    x = x[:,:,:500]

    print(x.shape)
    
    first_letter = file.split('/')[-1][0]

    # create output vector. If file begins with 'P', then y = [1,0], else y = [0,1], having the same dimension as x
    if first_letter == 'P':
        y = np.zeros((x.shape[0],2))
        y[:,0] = 1
    else:
        y = np.zeros((x.shape[0],2))
        y[:,1] = 1

    X.append(x)
    Y.append(y)

['../data/walk_instruction_raw/P_1.mat', '../data/walk_instruction_raw/P_10.mat', '../data/walk_instruction_raw/P_2.mat', '../data/walk_instruction_raw/P_3.mat', '../data/walk_instruction_raw/P_4.mat', '../data/walk_instruction_raw/P_5.mat', '../data/walk_instruction_raw/P_9.mat', '../data/walk_instruction_raw/S_1.mat', '../data/walk_instruction_raw/S_10.mat', '../data/walk_instruction_raw/S_2.mat', '../data/walk_instruction_raw/S_3.mat', '../data/walk_instruction_raw/S_4.mat', '../data/walk_instruction_raw/S_5.mat', '../data/walk_instruction_raw/S_9.mat']
(56, 14, 500)
(37, 14, 500)
(49, 14, 500)
(56, 14, 500)
(58, 14, 500)
(57, 14, 500)
(60, 14, 500)
(60, 14, 500)
(67, 14, 500)
(60, 14, 500)
(80, 14, 500)
(79, 14, 500)
(66, 14, 500)
(84, 14, 500)


In [11]:
f = open('../results/data_sizes.txt', 'a')

f.write('walk_instr_raw: ' + str(sum([x.shape[0] for x in X])) + '\n')

f.close()

In [12]:
kernels, chans, samples = 1, 14, 500

In [13]:

Accuracies = []
p_acc = []
s_acc = []

for index_1 in range(0, 7):
    for index_2 in range(7,14):
        # build training set from allfiles expect the ones with index_1 and index_2
        X_train = [X[i] for i in range(len(X)) if i != index_1 and i != index_2]
        Y_train = [Y[i] for i in range(len(Y)) if i != index_1 and i != index_2]

        # build test set from the files with index_1 and index_2
        X_test = [X[i] for i in range(len(X)) if i == index_1 or i == index_2]
        Y_test = [Y[i] for i in range(len(Y)) if i == index_1 or i == index_2]

        X_train = np.concatenate(X_train)
        Y_train = np.concatenate(Y_train)

        # convert data to NHWC (trials, channels, samples, kernels) format.
        X_train = X_train.reshape(X_train.shape[0], chans, samples, kernels)

        model = EEGNet(nb_classes=2, Chans=chans, Samples=samples, dropoutRate=0.5, kernLength=32, F1=8, D=2, F2=16, dropoutType='Dropout')

        # compile the model and set the optimizers
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

        # count number of parameters in the model
        numParams    = model.count_params()     

        # set a valid path for the system to record model checkpoints
        # checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1, save_best_only=True)

        class_weights = {0:1, 1:1}

        fitted_model = model.fit(X_train, Y_train, batch_size = 32, epochs = 50, verbose = 2, 
                                class_weight=class_weights)
        
        #open file to write results in
        f = open("../results/results_walk_instr_raw.txt", "a") 

        # make prediction on the 2 test files individually
        X_test_1 = X_test[0].reshape(X_test[0].shape[0], chans, samples, kernels)
        X_test_2 = X_test[1].reshape(X_test[1].shape[0], chans, samples, kernels)

        probs_1 = model.predict(X_test_1)
        probs_2 = model.predict(X_test_2)
        preds_1 = probs_1.argmax(axis = -1)
        preds_2 = probs_2.argmax(axis = -1)
        acc_1 = balanced_accuracy_score(Y_test[0].argmax(axis=-1), preds_1)
        acc_2 = balanced_accuracy_score(Y_test[1].argmax(axis=-1), preds_2)
        f.write("----------------------SELECTED FILES: %d and %d---------------------\n" % (index_1, index_2))
        f.write("Classification accuracy for P: %f \n" % (acc_1))
        f.write("Classification accuracy for S: %f \n" % (acc_2))

        p_acc.append(acc_1)
        s_acc.append(acc_2)

        # make prediction on the 2 test files together
        X_test = np.concatenate(X_test)
        X_test = X_test.reshape(X_test.shape[0], chans, samples, kernels)
        Y_test = np.concatenate(Y_test)


        probs       = model.predict(X_test)
        preds       = probs.argmax(axis = -1)
        acc         = balanced_accuracy_score(Y_test.argmax(axis=-1), preds)
        f.write("Classification accuracy for P and S: %f \n" % (acc))
        f.write("\n")

        f.close()

        Accuracies.append(acc)

print("Mean classification accuracy: %f " % (np.mean(Accuracies)))
    

Epoch 1/50
24/24 - 6s - loss: 0.6881 - accuracy: 0.5710 - 6s/epoch - 269ms/step
Epoch 2/50
24/24 - 3s - loss: 0.6614 - accuracy: 0.6029 - 3s/epoch - 135ms/step
Epoch 3/50
24/24 - 3s - loss: 0.6476 - accuracy: 0.6414 - 3s/epoch - 124ms/step
Epoch 4/50
24/24 - 3s - loss: 0.6237 - accuracy: 0.6707 - 3s/epoch - 129ms/step
Epoch 5/50
24/24 - 3s - loss: 0.6066 - accuracy: 0.6879 - 3s/epoch - 125ms/step
Epoch 6/50
24/24 - 3s - loss: 0.6009 - accuracy: 0.6985 - 3s/epoch - 124ms/step
Epoch 7/50
24/24 - 3s - loss: 0.5843 - accuracy: 0.7105 - 3s/epoch - 123ms/step
Epoch 8/50
24/24 - 3s - loss: 0.5748 - accuracy: 0.7264 - 3s/epoch - 122ms/step
Epoch 9/50
24/24 - 3s - loss: 0.5653 - accuracy: 0.7384 - 3s/epoch - 139ms/step
Epoch 10/50
24/24 - 3s - loss: 0.5307 - accuracy: 0.7596 - 3s/epoch - 131ms/step
Epoch 11/50
24/24 - 3s - loss: 0.5305 - accuracy: 0.7809 - 3s/epoch - 122ms/step
Epoch 12/50
24/24 - 3s - loss: 0.4966 - accuracy: 0.7862 - 3s/epoch - 122ms/step
Epoch 13/50
24/24 - 3s - loss: 0.4383

c:\Users\HTPDIR-Laptop1\eeg_net_env\Lib\site-packages\sklearn\metrics\_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\HTPDIR-Laptop1\eeg_net_env\Lib\site-packages\sklearn\metrics\_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


4/4 [==============================] - 0s 30ms/step
Epoch 1/50
24/24 - 6s - loss: 0.6928 - accuracy: 0.5402 - 6s/epoch - 253ms/step
Epoch 2/50
24/24 - 3s - loss: 0.6585 - accuracy: 0.5952 - 3s/epoch - 121ms/step
Epoch 3/50
24/24 - 3s - loss: 0.6443 - accuracy: 0.6475 - 3s/epoch - 128ms/step
Epoch 4/50
24/24 - 3s - loss: 0.6199 - accuracy: 0.6622 - 3s/epoch - 133ms/step
Epoch 5/50
24/24 - 3s - loss: 0.6054 - accuracy: 0.6836 - 3s/epoch - 124ms/step
Epoch 6/50
24/24 - 3s - loss: 0.6055 - accuracy: 0.7091 - 3s/epoch - 122ms/step
Epoch 7/50
24/24 - 3s - loss: 0.5867 - accuracy: 0.7131 - 3s/epoch - 122ms/step
Epoch 8/50
24/24 - 3s - loss: 0.5682 - accuracy: 0.7306 - 3s/epoch - 121ms/step
Epoch 9/50
24/24 - 3s - loss: 0.5461 - accuracy: 0.7507 - 3s/epoch - 123ms/step
Epoch 10/50
24/24 - 3s - loss: 0.5400 - accuracy: 0.7614 - 3s/epoch - 125ms/step
Epoch 11/50
24/24 - 3s - loss: 0.5203 - accuracy: 0.7748 - 3s/epoch - 124ms/step
Epoch 12/50
24/24 - 3s - loss: 0.4955 - accuracy: 0.7815 - 3s/epoc

c:\Users\HTPDIR-Laptop1\eeg_net_env\Lib\site-packages\sklearn\metrics\_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


5/5 [==============================] - 0s 26ms/step
Epoch 1/50
23/23 - 6s - loss: 0.6822 - accuracy: 0.5627 - 6s/epoch - 273ms/step
Epoch 2/50
23/23 - 3s - loss: 0.6408 - accuracy: 0.6594 - 3s/epoch - 139ms/step
Epoch 3/50
23/23 - 3s - loss: 0.6189 - accuracy: 0.6866 - 3s/epoch - 127ms/step
Epoch 4/50
23/23 - 3s - loss: 0.6050 - accuracy: 0.6975 - 3s/epoch - 126ms/step
Epoch 5/50
23/23 - 3s - loss: 0.5784 - accuracy: 0.7153 - 3s/epoch - 130ms/step
Epoch 6/50
23/23 - 3s - loss: 0.5502 - accuracy: 0.7561 - 3s/epoch - 123ms/step
Epoch 7/50
23/23 - 3s - loss: 0.5164 - accuracy: 0.7602 - 3s/epoch - 123ms/step
Epoch 8/50
23/23 - 3s - loss: 0.4975 - accuracy: 0.7875 - 3s/epoch - 126ms/step
Epoch 9/50
23/23 - 3s - loss: 0.4654 - accuracy: 0.8161 - 3s/epoch - 123ms/step
Epoch 10/50
23/23 - 3s - loss: 0.4183 - accuracy: 0.8501 - 3s/epoch - 122ms/step
Epoch 11/50
23/23 - 3s - loss: 0.3715 - accuracy: 0.8542 - 3s/epoch - 122ms/step
Epoch 12/50
23/23 - 3s - loss: 0.3667 - accuracy: 0.8556 - 3s/epoc

...

5/5 [==============================] - 0s 27ms/step
Epoch 1/50
24/24 - 6s - loss: 0.6903 - accuracy: 0.5532 - 6s/epoch - 263ms/step
Epoch 2/50
24/24 - 4s - loss: 0.6541 - accuracy: 0.6083 - 4s/epoch - 160ms/step
Epoch 3/50
24/24 - 3s - loss: 0.6341 - accuracy: 0.6205 - 3s/epoch - 141ms/step
Epoch 4/50
24/24 - 3s - loss: 0.6257 - accuracy: 0.6555 - 3s/epoch - 143ms/step
Epoch 5/50
24/24 - 3s - loss: 0.6130 - accuracy: 0.6729 - 3s/epoch - 141ms/step
Epoch 6/50
24/24 - 3s - loss: 0.6018 - accuracy: 0.6824 - 3s/epoch - 137ms/step
Epoch 7/50
24/24 - 3s - loss: 0.5776 - accuracy: 0.7268 - 3s/epoch - 140ms/step
Epoch 8/50
24/24 - 3s - loss: 0.5730 - accuracy: 0.7254 - 3s/epoch - 137ms/step
Epoch 9/50
24/24 - 3s - loss: 0.5484 - accuracy: 0.7429 - 3s/epoch - 143ms/step
Epoch 10/50
24/24 - 3s - loss: 0.5335 - accuracy: 0.7402 - 3s/epoch - 135ms/step
Epoch 11/50
24/24 - 3s - loss: 0.5141 - accuracy: 0.7672 - 3s/epoch - 136ms/step
Epoch 12/50
24/24 - 3s - loss: 0.4941 - accuracy: 0.7658 - 3s/epoc

c:\Users\HTPDIR-Laptop1\eeg_net_env\Lib\site-packages\sklearn\metrics\_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\HTPDIR-Laptop1\eeg_net_env\Lib\site-packages\sklearn\metrics\_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


4/4 [==============================] - 0s 38ms/step
Epoch 1/50
23/23 - 6s - loss: 0.6814 - accuracy: 0.5917 - 6s/epoch - 265ms/step
Epoch 2/50
23/23 - 3s - loss: 0.6627 - accuracy: 0.6441 - 3s/epoch - 148ms/step
Epoch 3/50
23/23 - 3s - loss: 0.6298 - accuracy: 0.6938 - 3s/epoch - 148ms/step
Epoch 4/50
23/23 - 3s - loss: 0.5974 - accuracy: 0.7076 - 3s/epoch - 137ms/step
Epoch 5/50
23/23 - 3s - loss: 0.5589 - accuracy: 0.7214 - 3s/epoch - 136ms/step
Epoch 6/50
23/23 - 3s - loss: 0.5131 - accuracy: 0.7434 - 3s/epoch - 140ms/step
Epoch 7/50
23/23 - 3s - loss: 0.4952 - accuracy: 0.7669 - 3s/epoch - 135ms/step
Epoch 8/50
23/23 - 3s - loss: 0.4746 - accuracy: 0.7641 - 3s/epoch - 137ms/step
Epoch 9/50
23/23 - 3s - loss: 0.4548 - accuracy: 0.7876 - 3s/epoch - 146ms/step
Epoch 10/50
23/23 - 3s - loss: 0.4435 - accuracy: 0.7821 - 3s/epoch - 145ms/step
Epoch 11/50
23/23 - 3s - loss: 0.4168 - accuracy: 0.7972 - 3s/epoch - 137ms/step
Epoch 12/50
23/23 - 3s - loss: 0.4050 - accuracy: 0.8193 - 3s/epoc

c:\Users\HTPDIR-Laptop1\eeg_net_env\Lib\site-packages\sklearn\metrics\_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\HTPDIR-Laptop1\eeg_net_env\Lib\site-packages\sklearn\metrics\_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


5/5 [==============================] - 0s 28ms/step
Mean classification accuracy: 0.840722 


In [14]:
# calculate the mean and std of the accuracies
p_acc = np.array(p_acc)
s_acc = np.array(s_acc)
acc = np.array(Accuracies)

f = open("../results/results_walk_instr_raw.txt", "a")
f.write("Mean overall accuracy: %f \n" % (np.mean(acc)))
f.write("Std overall accuracy: %f \n" % (np.std(acc)))
f.write("Mean P accuracy: %f \n" % (np.mean(p_acc)))
f.write("Std P accuracy: %f \n" % (np.std(p_acc)))
f.write("Mean S accuracy: %f \n" % (np.mean(s_acc)))
f.write("Std S accuracy: %f \n" % (np.std(s_acc)))
f.close()
